# Using Kqlmagic to analyze Azure SQL logs and metrics

Written by:  Julie Koesmarno

Original Notebook: [Using Kqlmagic to analyze Azure SQL logs and metrics](https://github.com/MsSQLGirl/jubilant-data-wizards/blob/main/Simple%20Demo/Parameterization/SQLDBLog/AzureSQLLogsAndMetricsWithLogAnalytics.ipynb)

Modifed by: Taiob  Ali  

  

There are two main tables in Azure Log Analytics (Azure Monitor Logs) workspace that capture Azure SQL events:

1. AzureDiagnostics
2. AzureMetric

## 1. Connect to Log Analytics workspace

Workspace is similar to what a database is to SQL. You connect to Log Analytics workspace to start querying data. 

### 1.1 Load Kqlmagic
Need to load Kqlmagic first before we can start writing queries.

In [ ]:
!pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
%reload_ext Kqlmagic

### 1.2 Connect to the desired Log Analytics workspace

In [ ]:
# Initialize workspace ID to connect to, to analyzie Azure SQL log data

workspaceID = '064d87f9-3f2c-47c4-99b3-bf84099d2ad5'

In [ ]:
%kql loganalytics://code;workspace=workspaceID;alias="Sqlalertdemo2" -try-azcli-login-by-profile

## 2. Analyze events by Diagnostic Settings

Let's do a simple query first to analyze the number of events by Operation Name. 

> **Note**: Each row in AzureDiagnostic represents an event for specific Operation or category. Some SQL actions may result in generating multiple events of different types.



In [ ]:
%%kql 
AzureDiagnostics
| summarize count() by OperationName


The above query's equivalent in SQL is:
```
SELECT COUNT(*) AS [count_]
FROM AzureDiagnostics
GROUP BY OperationName
```


Count my Azure SQL DB events by category / diagnostic settings.

In [ ]:
%%kql 
AzureDiagnostics
| where LogicalServerName_s == "sqlalertdemoserver"
| where TimeGenerated >= ago(5d)
| summarize count() by Category
| render barchart with (title = "Azure SQL DB Diagnostic Category")

## 3. Performance troubleshooting Query (from Azure Portal)

Potentially a query or deadlock on the system that could lead to poor performance. The following is a query suggested by Azure Portal.

In [ ]:
%%kql 
AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(1d)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName

# AzureMetrics

This is a sample query to dig into AzureMetrics

In [ ]:
%%kql 
AzureMetrics
| project-away TenantId, ResourceId, SubscriptionId, _ResourceId, ResourceGroup // Don't show sensitive columns :) 
| project TimeGenerated, MetricName, Total, Count, UnitName
| take 10


# AzureDiagnostics

This is a sample query to dig into AzureDiagnostics. This table tends to have more details than AzureMetrics.

In [ ]:
%%kql 
AzureDiagnostics
| project-away TenantId, ResourceId, SubscriptionId, ResourceGroup, _ResourceId // Don't show sensitive columns :) 
| take 10

## Analyze (non-audit) Events

In [ ]:
%%kql 
AzureDiagnostics
| summarize event_count=count() by bin(TimeGenerated, 2d), OperationName
| where OperationName <> "AuditEvent"
| render timechart 

## Deadlock Analysis

In [ ]:
%%kql 
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| project TimeGenerated, Category, Resource, OperationName, Type, deadlock_xml_s
| sort by TimeGenerated desc
| take 50

Find the deadlock query plan

In [ ]:
%%kql 
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| extend d = parse_xml(deadlock_xml_s)
| project TimeGenerated, QuerhPlanHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryplanhash"], QueryHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryhash"]
//| sort by TimeGenerated desc
| take 50

## Query Store Runtime Statistics Events

In [ ]:
%%kql 
AzureDiagnostics
| where OperationName == "QueryStoreRuntimeStatisticsEvent"
| project TimeGenerated, query_hash_s, statement_sql_handle_s, query_plan_hash_s
| take 50

## Analyze Errors

In [ ]:
%%kql 
AzureDiagnostics
| where OperationName == "ErrorEvent"
| extend ErrorNumber =  tostring(error_number_d) 
| summarize event_count=count() by EventTime = bin(TimeGenerated, 2d), ErrorNumber
| render timechart 

## Find Deleted table

In [ ]:
%%kql 
AzureDiagnostics
| where action_name_s in ('BATCH COMPLETED')
| project TimeGenerated, Category, action_name_s, statement_s
| where statement_s contains "DROP TABLE"
| sort by TimeGenerated desc 
| take 10

